In [1]:
import pigui
import importlib

importlib.reload(pigui)

<module 'pigui' from 'C:\\Users\\The Big G\\Documents\\Model-Identification\\pigui.py'>

In [2]:
system_list = [
    [[1],[100,10,1]],#]#,
    [[1],[10,1]],
    [[1],[1,1,1]],
    [[1],[10,1,1]],
    [[1],[1,10,1]],
    [[1],[1,1,10]]
] # placeholder systems for now
pigui.system_list = system_list

In [3]:
pigui.gui()

interactive(children=(Dropdown(description='Input', options=('Step', 'Rect', 'Doublet'), value='Step'), Dropdo…

<function pigui.sim(Input='Step', Noise='None', Technique='Genetic Algorithm')>